In [0]:
import numpy as np

# A python library for data observation and data analysis
import pandas as pd

# A Python library for Machine Learning
import sklearn
from sklearn import metrics # All kinds of metrics
from sklearn.model_selection import train_test_split # For splitting data in train and test sets
from sklearn.utils import shuffle, class_weight # Shuffling of data, calculator of class weights
from sklearn.model_selection import cross_val_score, GridSearchCV # Cross-validation score, grid search with cross-validation
from sklearn.preprocessing import StandardScaler, MinMaxScaler # Standardization, normalization

# A Python library to deal with imbalanced data
import imblearn
from imblearn.over_sampling import SMOTE # SMOTE algorithm for oversampling
from imblearn.under_sampling import RandomUnderSampler # Random undersampling
from imblearn.pipeline import Pipeline # To combine undersampling/overampling with grid search

# For counting elements in a dictionary
import collections
from collections import Counter

Core model zoo

In [0]:
'''
    Core model zoo:
    - XGBoost: optimized and scalable gradient tree boosting library
    - LightGBM: gradient boosting framework using decision trees
    - Multi-layer perceptron: a simple design of an artificial neuron network (ANN)
    - K-nearest neighbors: classification through finding of k nearest neighbors of each input
'''

# XGBoost Classifier
import xgboost as xgb 
# LightGBM Classifier
from lightgbm import LGBMClassifier 
# Multi-layer Perceptron
from sklearn.neural_network import MLPClassifier 
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

In [25]:
# Mount Colab with local drive to access data
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Load train data and labels for each timepoint

In [0]:
def load_data(n_groups=4):
  '''
    Load training for all four timepoints (only the free latter were used)

    Returns: training data, labels, unlabeled data and metadata relative to unlabeled data for each timepoint
  '''

  x_groups = [] # training data
  y_groups = [] # training labels
  t_groups = [] # unlabeled data
  m_groups = [] # metadata relative to unlabeled data (i.e., well and plate number) --> used for predicting unlabeled data

  for i in range(n_groups):
    train_fname = 'gdrive/My Drive/ML2/data/labeled_' + repr(i) + '.csv'

    # Collect training data for each timepoint as a NumPy array
    x_groups.append(np.asarray(pd.read_csv(train_fname))) 

    label_fname = 'gdrive/My Drive/ML2/data/labels_' + repr(i) + '.csv'
    label_group = pd.read_csv(label_fname)

    # Collect labels for each timepoint as a NumPy array
    y_groups.append(np.reshape(np.asarray(label_group), (len(label_group),)))

    # Check that train data and labels have same dimensions
    assert(x_groups[i].shape[0] == y_groups[i].shape[0])
    print('OK. Train data and labels at timepoint ' + repr(i) + ' have same x-dimensions (' + repr(x_groups[i].shape[0]) + ').')

    # Collect unlabeled data for each timepoint as a NumPy array
    test_fname = 'gdrive/My Drive/ML2/data/unlabeled_' + repr(i) + '.csv'
    t_groups.append(np.asarray(pd.read_csv(test_fname)))

    print('Unlabeled data at timepoint ' + repr(i) + ': ' + repr(t_groups[i].shape[0]))

    '''# Collect metadata relative to unlabeled data for each timepoint as a NumPy array
    meta_fname = 'gdrive/My Drive/ML2/data/meta_unlabeled_' + repr(i) + '.csv'
    m_groups.append(np.asarray(pd.read_csv(meta_fname)))'''
  
  return x_groups, y_groups, t_groups

In [27]:
x_groups, y_groups, t_groups = load_data()

OK. Train data and labels at timepoint 0 have same x-dimensions (41392).
Unlabeled data at timepoint 0: 288156
OK. Train data and labels at timepoint 1 have same x-dimensions (45138).
Unlabeled data at timepoint 1: 308990
OK. Train data and labels at timepoint 2 have same x-dimensions (47781).
Unlabeled data at timepoint 2: 321565
OK. Train data and labels at timepoint 3 have same x-dimensions (51846).
Unlabeled data at timepoint 3: 340905


Grid search (not used until now)

In [0]:
def grid_search(i, n_folds=5, method='lgbm', smote=False, rus=False):
  '''
    Grid search combined with 5-fold cross-validation
    i: timepoint number
    n_folds: number of folds for cross-validation
    method: ML algorithm to test (LGBM, XGBoost, MLP, kNN)
    smote: Perform grid search with oversampling using SMOTE
    rus: Perform grid search with undersampling using random undersampling

    return: optimal hyperparameters, merged with default parameters
  '''

  # Split labeled data in train and test set
  print('Splitting data...')
  X_train, X_test, Y_train, Y_test = train_test_split(x_groups[i], y_groups[i], test_size=0.1, random_state=42)
  class_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train) # Get class weights

  # Dimension of Train and Test set 
  print("Dimension of Train set:", X_train.shape)
  print("Dimension of Test set:", X_test.shape)

  # Standardization of labeled data
  print('Preprocessing...')
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  if method == 'lgbm':
    ''' 
      Parameter grid for LGBM:
      num_leaves: maximum of number of leaves in a tree
      n_estimators: number of boosting iterations
      reg_alpha: L1-regularization term
      reg_beta: L2-regularization term
    '''

    '''
      Default parameters for LGBM:
      boosting_type: Gradient boosting with decision trees (GBDT)
      objective: multiclass classification
      class_weight: balanced
      random_state: to preserve same results for each run
    '''

    default_param = {'boosting_type':'gbdt', 'objective':'multiclass', 
                       'class_weight':'balanced', 'random_state': 42} 

    if smote:
      # Create pipeline if the user wants to test SMOTE with grid search
      print('Grid search will be performed with SMOTE.')
      
      model = Pipeline([
        ('sampling', SMOTE()),
        ('classification', LGBMClassifier())
      ])
      params_grid = {'classification__num_leaves':[10, 31, 50], 'classification__n_estimators':[50, 100, 200], 
                 'classification__reg_alpha':[0, 0.1], 'classification__reg_beta':[0, 0.1]}
    
    elif rus:
      # Create pipeline if the user wants to test random undersampling with grid search
      print('Grid search will be performed with random undersampling.')

      model = Pipeline([
        ('sampling', RandomUnderSampler(random_state=42)),
        ('classification', LGBMClassifier(**default_param))
      ])
      params_grid = {'classification__num_leaves':[10, 31, 50], 'classification__n_estimators':[50, 100, 200], 
                 'classification__reg_alpha':[0, 0.1], 'classification__reg_beta':[0, 0.1]}

    else:
      model = LGBMClassifier(**default_param)
    
      params_grid = {'num_leaves': [10, 31, 50], 'n_estimators':[50, 100, 200], 
                     'reg_alpha':[0, 0.1], 'reg_beta':[0, 0.1]}
  
  elif method == 'xgb':
    ''' 
      Parameter grid for XGBoost:
      max_depth: maximum tree depth
      min_child_weight: minimum sum of instance weight needed in a child (leaf).
      subsample: subsample ratio of the training instances
      lambda: L2-regularization term
    '''

    '''
      Default parameters for XGBoost:
      objective: multiclass classification with softmax
      tree_method, gpu_id: methods to use XGBoost using Colab's GPU
      sample_weights: class weights (to account for imbalanced dataset)
    '''

    default_param = {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 
                     'gpu_id': '0', 'sample_weights': class_weights}

    model = xgb.XGBClassifier(**default_param)

    params_grid = {'max_depth':[3, 6, 9], 'min_child_weight':[0.25, 0.5, 1], 
                   'subsample':[0.5, 1], 'lambda':[0.1, 1, 10]}
  
  elif method == 'mlp':
    ''' 
      Parameter grid for MLP:
      hidden_layer_sizes: size of hidden layers
      solver: solver for weight optimization (Adam or LBFGS)
      alpha: L2-regularization term
      activation: activation function or each layer (ReLU or tanh)
    '''

    '''
      Default parameters for MLP:
      max_iter: maximum number of iterations, to let the method converge
    '''
    default_param = {'max_iter':1000}

    model = MLPClassifier(**default_param)

    params_grid={'hidden_layer_sizes':[50, 100, 200], 'solver':['adam', 'lbfgs'], 'alpha':[0.01, 0.001, 0.0001], 'activation':['relu', 'tanh']}
  
  elif method == 'knn':
    ''' 
      Parameter grid for kNN:
      n_neigbors: number of neighbors of given input to consider
      weights: weight function used in prediction
      metric: distance metric used to build the tree
    '''
    default_param = {'algorithm': 'auto'}
    model = KNeighborsClassifier(**default_param)

    params_grid={'n_neighbors':[3, 5, 10, 20, 50, 100], 'weights':['distance', 'uniform'], 'metric':['euclidean', 'manhattan']}
  else:
    raise NameError('Unknown method.')

  print('Training...')
  # Grid search with 5-fold cross-validation
  grid_search_model = GridSearchCV(model, params_grid, cv=n_folds, verbose=5, n_jobs=-1)
  grid_search_model.fit(X_train_scaled, Y_train)

  # Print best cross-validation score found using grid search
  print('Best score:', grid_search_model.best_score_,"\n") 

  # View optimal hyperparameters found using grid search
  print('Best params:', grid_search_model.best_params_)

  return {**default_param, **grid_search_model.best_params_}

In [29]:
# Uncomment to perform grid search with 5-fold cross-validation (duration: 60-120 min):
'''
best_params = []
n_groups = 4
for i in range(1, n_groups):
  best_params.append(grid_search(i, method='lgbm'))
'''

"\nbest_params = []\nn_groups = 4\nfor i in range(1, n_groups):\n  best_params.append(grid_search(i, method='lgbm'))\n"

Grid search solutions

In [0]:
# GridSearhCV solutions for each method and each timepoint

##### LGBM Grid search results (no resampling, oversampling with SMOTE, undersampling with RUS)
default_lgbm_param = {'boosting_type':'gbdt', 'objective':'multiclass', 
                       'class_weight':'balanced', 'random_state': 42} 

best_lgbm_params = [{**default_lgbm_param, 'n_estimators':200, 'num_leaves':50, 'reg_alpha':0.1, 'reg_beta':0}, 
                    {**default_lgbm_param, 'n_estimators':200, 'num_leaves':50, 'reg_alpha':0, 'reg_beta':0}, 
                    {**default_lgbm_param, 'n_estimators':200, 'num_leaves':50, 'reg_alpha':0, 'reg_beta':0}, 
                   ]

best_smote_param = [{**default_lgbm_param, 'n_estimators': 200, 'num_leaves': 50, 'reg_alpha': 0, 'reg_beta': 0},
                    {**default_lgbm_param, 'n_estimators': 200, 'num_leaves': 50, 'reg_alpha': 0, 'reg_beta': 0},
                    {**default_lgbm_param, 'n_estimators': 200, 'num_leaves': 50, 'reg_alpha': 0, 'reg_beta': 0}
                   ]

best_rus_params = [{**default_lgbm_param, 'n_estimators': 50, 'num_leaves': 10, 'reg_alpha': 0, 'reg_beta': 0},
                   {**default_lgbm_param, 'n_estimators': 50, 'num_leaves': 10, 'reg_alpha': 0, 'reg_beta': 0},
                   {**default_lgbm_param, 'n_estimators': 50, 'num_leaves': 10, 'reg_alpha': 0, 'reg_beta': 0}
                  ]

##### XGB Grid search results
default_xgb_param = {'objective': 'multi:softmax', 'tree_method': 'gpu_hist', 'gpu_id': '0'}

best_xgb_params = [{**default_xgb_param, 'max_depth': 9, 'min_child_weight': 0.5, 'subsample': 1, 'lambda':0.1}, 
                    {**default_xgb_param, 'max_depth': 9, 'min_child_weight': 0.25, 'subsample': 1, 'lambda':0.1}, 
                    {**default_xgb_param, 'max_depth': 9, 'min_child_weight': 0.25, 'subsample': 1, 'lambda':0.1}, 
                   ]

##### kNN Grid search results
default_knn_param = {'algorithm':'auto'}


best_knn_params = [{**default_knn_param, 'n_neighbors': 5, 'distance': 'manhattan', 'weights':'distance'}, 
                   {**default_knn_param, 'n_neighbors': 5, 'distance': 'manhattan', 'weights':'distance'}, 
                   {**default_knn_param, 'n_neighbors': 3, 'distance': 'manhattan', 'weights':'distance'}, 
                  ]

##### MLP Grid search results
default_mlp_param = {'max_iter':1000}

best_mlp_params = [{**default_mlp_param, 'hidden_layer_sizes':200, 'solver': 'adam', 'activation': 'relu', 'alpha':0.001}, 
                   {**default_mlp_param, 'hidden_layer_sizes':200,'solver': 'adam', 'activation': 'relu', 'alpha':0.01}, 
                   {**default_mlp_param, 'hidden_layer_sizes':200, 'solver': 'adam', 'activation': 'relu', 'alpha':0.001}, 
                  ]


Training method

In [0]:
def train(i, method, model_params, smote=False, rus=False):
  '''
    i: timepoint number [0, 1 or 2]
    smote: boolean to use SMOTE (if True)
    rus: boolean to use random undersampling (if True)

    returns: model used, tested parameters and predictions on unlabeled dataset
  '''

  print('---------- Timepoint ' + repr(i+1) + ' ----------')
  
  # Split train and test set using 85% of labeled data
  print('Splitting data...')
  X_train, X_test, Y_train, Y_test = train_test_split(x_groups[i+1], y_groups[i+1], test_size=0.15, random_state=42)
  X_unlabeled = t_groups[i+1]

  # Dimension of Train and Test set 
  print("Dimension of Train set", X_train.shape)
  print("Dimension of Test set", X_test.shape,"\n")

  # Standardization
  print('Preprocessing...')
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)
  X_unlabeled_scaled = scaler.transform(X_unlabeled)

  # Compute class weights (used in some of our ML methods)
  class_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)

  if smote:
    # Oversampling with SMOTE
    smote_ = SMOTE()
    X_train_scaled, Y_train = smote_fit.sample(X_train_scaled, Y_train)
  elif rus:
    # Undersampling with random undersampling
    rus_ = RandomUnderSampler(random_state=42)
    X_train_scaled, Y_train = rus_.fit_resample(X_train_scaled, Y_train)
  
  param_lgbm = {'boosting_type': 'gbdt','objective': 'multiclass', 'random_state': 42, 'n_estimators': 200, 
                'num_leaves': 50, 'reg_alpha': 0.1, 'reg_beta': 0}

  print('Training...')
  # Build model with inputed parameters
  if method == 'lgbm':
    model = LGBMClassifier(**param_lgbm)
  elif method == 'xgb':
    model_params.update({'sample_weights': class_weights})
    model = xgb.XGBClassifier(**model_params)
  elif method == 'mlp':
    model = MLPClassifier(**model_params)
  elif method == 'knn':
    model = KNeighborsClassifier(**model_params)
  else:
    raise NameError('Unknown method.')

  # Print model for user 
  print(model)

  # Train model
  model.fit(X_train_scaled, Y_train)

  print('Predicting...')
  # Predict test sest
  Y_pred = model.predict(X_test_scaled)

  # Print confusion matrix
  print('Confusion matrix')
  print(metrics.confusion_matrix(Y_test,Y_pred))
  print("\n") 

  # Print evaluation metrics
  print('MCC score:', sklearn.metrics.matthews_corrcoef(Y_test, Y_pred))
  print("Training set score: %f" % model.score(X_train_scaled, Y_train))
  print("Testing  set score: %f" % model.score(X_test_scaled, Y_test ))

  # Give predictions for unlabeled dataset
  print('Predicting unlabeled test set...')
  pred = model.predict(X_unlabeled_scaled)
  
  return Counter(pred)

Train for each timepoint

In [33]:
n_groups = 4
for i in range(n_groups-1):
  unlabeled_pred = train(i, 'lgbm', best_lgbm_params[i])
  # Print unlabeled predictions as a Counter 
  # A counter keeps track of how many times equivalent values are found in an array
  print(unlabeled_pred)

---------- Timepoint 1 ----------
Splitting data...
Dimension of Train set (38367, 94)
Dimension of Test set (6771, 94) 

Preprocessing...
Training...
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=200, n_jobs=-1, num_leaves=50,
               objective='multiclass', random_state=42, reg_alpha=0.1,
               reg_beta=0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
Predicting...
Confusion matrix
[[1096    2   29    4   25    6    0    3   52   52]
 [  28  203    2    6   11    9    0    0    9    4]
 [  89    4  536    1   12    0    0    3   75   80]
 [  42    5    1  159    1    4    0    0    9    4]
 [  43    2   20    2  988    0    3    1   37   66]
 [  29    2    3    0    4  254    2    0    8   12]
 [   2  